### использование tf-idf для вытягивание сущностей из чанков и документов

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
loader = Docx2txtLoader("/Applications/programming/kg_llm/assets/скан-архив/install_and_settings.docx")
docs = loader.load()

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [4]:
# splits это наши чанки 

In [5]:
text_chunks =[doc.page_content for doc in splits]

In [6]:
from src.graph_chunks.splitter import GrapSplitterTfIdf

[nltk_data] Downloading package stopwords to /Users/kodin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/kodin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
spltc = GrapSplitterTfIdf()

In [8]:
spltc.split_documents(docs)

[Document(page_content='«Скан-Загрузка документов». Руководство пользователя\n\n10\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nСКАН-ЗАГРУЗКА\n\nдокументов\n\nРуководство по установке и настройке\n\n\n\n\n\n\n\n\n\n\n\nРостов-на-Дону 2022\n\n\n\n\n\n\t\tВведение\n\nДанный документ описывает порядок подготовки к работе и использования программного продукта\n\n«Скан-Загрузка документов» (далее – СЗД). Инструкция относится к релизу CЗД: 3.0.1.0.\n\nСписок терминов и сокращений, использованных в данном документе:\n\n\n\n\t«Скан-Загрузка документов» (СЗД) — программный продукт «Скан-Загрузка документов».\n\n\tТерминальный режим – режим работы с базой 1С:Предприятие через подключение к удалённому рабочему столу сервера (средство «Подключение к удаленному рабочему столу», входит в стандартные программы Windows).\n\n\tПФ — печатная форма.\n\n\tТПФ – тип печатной формы (например, «ТОРГ-12»).\n\n\t«Скан-Архив» (СА) —программный продукт, предназначенный для автоматизации создани

## Удаление стоп слов с чанков

In [198]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

# Загрузка стоп-слов и пунктуации
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('russian'))
punctuation = set(string.punctuation)

[nltk_data] Downloading package stopwords to /Users/kodin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/kodin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [199]:
def preprocess_text(text):
    words = word_tokenize(text.lower())  # Привести к нижнему регистру и токенизировать
    filtered_words = [word for word in words if word not in stop_words and word not in punctuation]
    return " ".join(filtered_words)

In [200]:
prprocess_text_chunks = [preprocess_text(text) for text in text_chunks]

## Приведение к нижнему регистру и лемматизация

In [201]:
import spacy

# Загрузка языковой модели spaCy
nlp = spacy.load("ru_core_news_lg")

def preprocess_and_lemmatize(text):
    doc = nlp(text.lower())  # Привести к нижнему регистру и лемматизировать
    lemmatized_words = [token.lemma_ for token in doc if token.text not in punctuation and token.text not in stop_words]
    return " ".join(lemmatized_words)

In [202]:
lemmatized_text_chunks = [preprocess_and_lemmatize(text) for text in prprocess_text_chunks]

In [203]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Пример текстовых данных
documents = lemmatized_text_chunks

# Создание объекта TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3))

# Применение TF-IDF к текстовым данным
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

# Определение порога для отбора ключевых слов
threshold = 0.15
important_words = [word for word, score in zip(tfidf_vectorizer.get_feature_names_out(), tfidf_matrix.toarray()[0]) if score >= threshold]
print("Важные слова:", important_words)

Важные слова: ['документ', 'документ руководство', 'загрузка', 'загрузка документ', 'загрузка документ руководство', 'руководство', 'скан загрузка', 'скан загрузка документ']


In [204]:
keywords=[]
for row in tfidf_matrix:
    # Сопоставляем слова с их TF-IDF весами
    word_scores = {word: score for word, score in zip(tfidf_vectorizer.get_feature_names_out(), row.toarray().flatten())}
    
    # Сортировка слов в документе по весам
    sorted_words = sorted(word_scores.items(), key=lambda item: item[1], reverse=True)
    
    # Берем, например, топ-3 слова с наивысшим весом TF-IDF в качестве ключевых слов
    document_keywords = [word for word, score in sorted_words[:3]]
    
    # Добавляем список ключевых слов для данного документа
    keywords.append(document_keywords)

print(keywords)

[['документ руководство', 'загрузка документ руководство', 'документ'], ['скан загрузка', 'скан загрузка документ', 'подключение удалённый'], ['архив', 'печатный', 'скан'], ['обработать', 'обработать скан', 'скан'], ['сервис', '1с', 'облачный'], ['печатный документ', 'документ база', 'документ'], ['документ база', 'документ', 'которые'], ['epf', 'обработка', 'epf версия'], ['прикреплённых', 'epf', 'просмотр'], ['брс', 'скан брс', 'сохранение'], ['брс', 'скан брс', 'office'], ['office', 'файл', 'libre office сзд'], ['модуль', 'форма модуль', 'epf'], ['gendalf ru технический', 'ru технический', 'gendalf ru'], ['платформа', 'режим управляемого', 'режим управляемого приложение'], ['редакция', 'управление', 'предприятие'], ['бухгалтерия', 'редакция', 'бгу'], ['windows', 'framework', 'net'], ['режим удалённый', 'удалённый', 'remoteapp'], ['500', '500 мб', 'мб'], ['версия windows', 'разрядная', 'гб'], ['сканер', 'необходимый сканер', 'либо'], ['формат', 'образ', 'скан образ'], ['pdf', 'pdf до

In [205]:
len(keywords)

142

In [206]:
for text, keyw in zip(splits, keywords):
    print(text)
    print(keyw)
    print()

page_content='«Скан-Загрузка документов». Руководство пользователя\n\n10\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nСКАН-ЗАГРУЗКА\n\nдокументов\n\nРуководство по установке и настройке\n\n\n\n\n\n\n\n\n\n\n\nРостов-на-Дону 2022\n\n\n\n\n\n\t\tВведение\n\nДанный документ описывает порядок подготовки к работе и использования программного продукта\n\n«Скан-Загрузка документов» (далее – СЗД). Инструкция относится к релизу CЗД: 3.0.1.0.\n\nСписок терминов и сокращений, использованных в данном документе:' metadata={'source': '/Applications/programming/kg_llm/assets/скан-архив/install_and_settings.docx'}
['документ руководство', 'загрузка документ руководство', 'документ']

page_content='«Скан-Загрузка документов» (далее – СЗД). Инструкция относится к релизу CЗД: 3.0.1.0.\n\nСписок терминов и сокращений, использованных в данном документе:\n\n\n\n\t«Скан-Загрузка документов» (СЗД) — программный продукт «Скан-Загрузка документов».\n\n\tТерминальный режим – режим работы с базо

In [207]:
import os
from langchain_community.graphs import Neo4jGraph
from langchain_core.documents.base import Document
from langchain_community.graphs.graph_document import GraphDocument, Node,Relationship

os.environ["NEO4J_URI"] ="bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "kodin"
os.environ["NEO4J_PASSWORD"] = "12345678"
graph = Neo4jGraph(database='chank-keyword')

In [208]:
import hashlib
import secrets

def get_random_hash():
    random_bytes = secrets.token_bytes(64)
    hash_object = hashlib.sha256(random_bytes)
    return hash_object.hexdigest()

In [209]:
nodes = []
relationships=[]
for text, keyw in zip(splits, keywords):
        src = Node(id= get_random_hash(), 
                type='chunk', 
                properties={'text':  text.page_content})
        nodes.append(src)
        for k in keyw:
            target = Node(id=k, 
                        type='entity', 
                        properties={'text': k})
            nodes.append(target)
            relationships.append(
                Relationship(source=src, 
                            target=target, 
                            type='contain_keyword'))
            

In [210]:
source = Document(page_content="short_content", metadata={"path": 'pa'})

In [211]:
graph_doc = GraphDocument(nodes=nodes, relationships=relationships, source=source)

In [212]:
    
graph.add_graph_documents(
    [graph_doc],
    baseEntityLabel=True,
    include_source=True
)

# посмотрим ключевые слова по документам

In [ ]:
docs=[]

In [221]:
loader = Docx2txtLoader("/Applications/programming/kg_llm/assets/скан-архив/install_and_settings.docx")
docs += loader.load()
loader = Docx2txtLoader("/Applications/programming/kg_llm/assets/dozor/install.docx")
docs += loader.load()
loader = Docx2txtLoader("/Applications/programming/kg_llm/assets/dozor/install.docx")
docs += loader.load()
loader = Docx2txtLoader("/Applications/programming/kg_llm/assets/dozor/usage_big.docx")
docs += loader.load()
loader = Docx2txtLoader("/Applications/programming/kg_llm/assets/dozor/usage.docx")
docs += loader.load()
loader = Docx2txtLoader("/Applications/programming/kg_llm/assets/other/гендальф.docx")
docs += loader.load()
loader = Docx2txtLoader("/Applications/programming/kg_llm/assets/other/гендальф.docx")
docs += loader.load()
loader = Docx2txtLoader("/Applications/programming/kg_llm/assets/other/гендальф.docx")
docs += loader.load()

In [222]:
text_chunks =[doc.page_content for doc in docs]
prprocess_text_chunks = [preprocess_text(text) for text in text_chunks]

In [223]:
lemmatized_text_chunks = [preprocess_and_lemmatize(text) for text in prprocess_text_chunks]

In [240]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Пример текстовых данных
documents = lemmatized_text_chunks

# Создание объекта TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(2, 4))

# Применение TF-IDF к текстовым данным
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

# Определение порога для отбора ключевых слов
threshold = 0.15
important_words = [word for word, score in zip(tfidf_vectorizer.get_feature_names_out(), tfidf_matrix.toarray()[0]) if score >= threshold]
print("Важные слова:", important_words)

Важные слова: ['печатный форма']


In [242]:
keywords=[]
for row in tfidf_matrix:
    # Сопоставляем слова с их TF-IDF весами
    word_scores = {word: score for word, score in zip(tfidf_vectorizer.get_feature_names_out(), row.toarray().flatten())}
    
    # Сортировка слов в документе по весам
    sorted_words = sorted(word_scores.items(), key=lambda item: item[1], reverse=True)
    
    # Берем, например, топ-3 слова с наивысшим весом TF-IDF в качестве ключевых слов
    document_keywords = [word for word, score in sorted_words[:7]]
    
    # Добавляем список ключевых слов для данного документа
    keywords.append(document_keywords)

print(keywords)

[['печатный форма', 'программный компонент', 'загрузка документ', 'рабочий место', 'вариант печатный', 'вариант печатный форма', 'дополнительный отчёт'], ['печатный форма', 'программный компонент', 'загрузка документ', 'рабочий место', 'вариант печатный', 'вариант печатный форма', 'дополнительный отчёт'], ['рабочий станция', 'рисунок рис', 'alt linux', 'безопасность рабочий', 'безопасность рабочий станция', 'действительный лицензия', 'рис нажатие'], ['рабочий станция', 'рисунок рис', 'alt linux', 'безопасность рабочий', 'безопасность рабочий станция', 'действительный лицензия', 'рис нажатие'], ['рабочий станция', 'безопасность рабочий', 'безопасность рабочий станция', 'запретить сайт', 'результат проверка', 'количество сайт', 'отчёт результат'], ['рабочий станция', 'alt linux', 'безопасность рабочий', 'безопасность рабочий станция', 'запретить сайт', 'результат проверка', 'количество сайт'], ['автоматический архивирование', 'разностный архив', 'каталог данных', 'раскрытие архив', '1с п

In [243]:
for text, keyw in zip(docs, keywords):
    print(text.page_content[:10])
    print(keyw)
    print()

«Скан-Загр
['печатный форма', 'программный компонент', 'загрузка документ', 'рабочий место', 'вариант печатный', 'вариант печатный форма', 'дополнительный отчёт']

«Скан-Загр
['печатный форма', 'программный компонент', 'загрузка документ', 'рабочий место', 'вариант печатный', 'вариант печатный форма', 'дополнительный отчёт']

«Дозор. Мо
['рабочий станция', 'рисунок рис', 'alt linux', 'безопасность рабочий', 'безопасность рабочий станция', 'действительный лицензия', 'рис нажатие']

«Дозор. Мо
['рабочий станция', 'рисунок рис', 'alt linux', 'безопасность рабочий', 'безопасность рабочий станция', 'действительный лицензия', 'рис нажатие']

«Дозор. Мо
['рабочий станция', 'безопасность рабочий', 'безопасность рабочий станция', 'запретить сайт', 'результат проверка', 'количество сайт', 'отчёт результат']

«Дозор. Мо
['рабочий станция', 'alt linux', 'безопасность рабочий', 'безопасность рабочий станция', 'запретить сайт', 'результат проверка', 'количество сайт']

2	Хранител
['автоматический ар